<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Refs" data-toc-modified-id="Refs-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Refs</a></span></li><li><span><a href="#Import-packages" data-toc-modified-id="Import-packages-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Import packages</a></span></li><li><span><a href="#Get-data" data-toc-modified-id="Get-data-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Get data</a></span></li><li><span><a href="#Beam-IO" data-toc-modified-id="Beam-IO-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Beam IO</a></span><ul class="toc-item"><li><span><a href="#Read-from-bigquery-and-write-to-parquet-in-GCS" data-toc-modified-id="Read-from-bigquery-and-write-to-parquet-in-GCS-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Read from bigquery and write to parquet in GCS</a></span></li></ul></li><li><span><a href="#Beam-transformations" data-toc-modified-id="Beam-transformations-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Beam transformations</a></span><ul class="toc-item"><li><span><a href="#Aggregations" data-toc-modified-id="Aggregations-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Aggregations</a></span><ul class="toc-item"><li><span><a href="#Group-by-and-count" data-toc-modified-id="Group-by-and-count-5.1.1"><span class="toc-item-num">5.1.1&nbsp;&nbsp;</span>Group by and count</a></span></li><li><span><a href="#Filter-and-then-count" data-toc-modified-id="Filter-and-then-count-5.1.2"><span class="toc-item-num">5.1.2&nbsp;&nbsp;</span>Filter and then count</a></span></li><li><span><a href="#Word-count-then-sort" data-toc-modified-id="Word-count-then-sort-5.1.3"><span class="toc-item-num">5.1.3&nbsp;&nbsp;</span>Word count then sort</a></span></li></ul></li><li><span><a href="#Join:-CoGroupByKey" data-toc-modified-id="Join:-CoGroupByKey-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Join: CoGroupByKey</a></span></li><li><span><a href="#Partition" data-toc-modified-id="Partition-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>Partition</a></span></li><li><span><a href="#Flatten" data-toc-modified-id="Flatten-5.4"><span class="toc-item-num">5.4&nbsp;&nbsp;</span>Flatten</a></span></li><li><span><a href="#Composite-transformation" data-toc-modified-id="Composite-transformation-5.5"><span class="toc-item-num">5.5&nbsp;&nbsp;</span>Composite transformation</a></span></li></ul></li><li><span><a href="#ParDo-transformations" data-toc-modified-id="ParDo-transformations-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>ParDo transformations</a></span><ul class="toc-item"><li><span><a href="#Pardo-for-Map:-Housing-data" data-toc-modified-id="Pardo-for-Map:-Housing-data-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Pardo for Map: Housing data</a></span></li><li><span><a href="#ParDo-for-FlatMap:-Wordcount-revisit" data-toc-modified-id="ParDo-for-FlatMap:-Wordcount-revisit-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>ParDo for FlatMap: Wordcount revisit</a></span></li></ul></li><li><span><a href="#Side-inputs-and-side-outputs-in-ParDo" data-toc-modified-id="Side-inputs-and-side-outputs-in-ParDo-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Side inputs and side outputs in ParDo</a></span><ul class="toc-item"><li><span><a href="#Side-inputs-in-Pardo" data-toc-modified-id="Side-inputs-in-Pardo-7.1"><span class="toc-item-num">7.1&nbsp;&nbsp;</span>Side inputs in Pardo</a></span></li><li><span><a href="#Side-outputs-in-ParDo" data-toc-modified-id="Side-outputs-in-ParDo-7.2"><span class="toc-item-num">7.2&nbsp;&nbsp;</span>Side outputs in ParDo</a></span></li></ul></li></ul></div>

# Refs

Beam dataframe colab notebook
https://colab.research.google.com/github/apache/beam/blob/master/examples/notebooks/tour-of-beam/dataframes.ipynb#scrollTo=YWYVFkvFuksz

Udemy course
https://www.udemy.com/course/apache-beam/

Good doc
https://beam.apache.org/documentation/transforms/python/elementwise/flatmap/



# Import packages 

In [9]:
import apache_beam as beam
import pandas as pd
import pandas_gbq as gbq
import pyarrow.parquet
# from pandas_profiling import ProfileReport
from apache_beam.options.pipeline_options import PipelineOptions

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Get data
`../data/housing_short.csv` is created based on "california housing price" dataset from Kaggle (https://www.kaggle.com/datasets/camnugent/california-housing-prices.

`../data/housing_short.csv` is a shuffled short version of the original dataset.

In [223]:
!head ../data/housing_short.csv


longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
-121.7,36.6,19.0,3562.0,530.0,1607.0,505.0,5.0162,283100.0,<1H OCEAN
-118.3,34.01,48.0,4217.0,1095.0,3298.0,949.0,1.9152,122300.0,<1H OCEAN
-121.75,36.92,46.0,1362.0,321.0,1068.0,305.0,2.4615,177800.0,<1H OCEAN
-117.92,33.92,19.0,2181.0,400.0,1272.0,337.0,5.1952,302100.0,<1H OCEAN
-122.05,37.36,34.0,2400.0,419.0,1017.0,384.0,4.1369,316900.0,<1H OCEAN
-118.86,34.16,16.0,1509.0,216.0,578.0,235.0,10.2614,410800.0,<1H OCEAN
-117.96,33.99,25.0,1348.0,210.0,660.0,200.0,5.2852,297600.0,<1H OCEAN
-118.16,34.12,38.0,2231.0,489.0,940.0,484.0,5.4165,435100.0,<1H OCEAN
-117.02,32.71,30.0,3187.0,592.0,2082.0,631.0,3.5388,118500.0,NEAR OCEAN


In [224]:
df = pd.read_csv('../data/housing_short.csv')
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           120 non-null    float64
 1   latitude            120 non-null    float64
 2   housing_median_age  120 non-null    float64
 3   total_rooms         120 non-null    float64
 4   total_bedrooms      118 non-null    float64
 5   population          120 non-null    float64
 6   households          120 non-null    float64
 7   median_income       120 non-null    float64
 8   median_house_value  120 non-null    float64
 9   ocean_proximity     120 non-null    object 
dtypes: float64(9), object(1)
memory usage: 9.5+ KB


In [12]:
df.groupby('ocean_proximity').size()


ocean_proximity
<1H OCEAN     9136
INLAND        6551
ISLAND           5
NEAR BAY      2290
NEAR OCEAN    2658
dtype: int64

# Beam IO
## Read from bigquery and write to parquet in GCS

In [16]:
query = """
SELECT county_name, county_fips_code, state_name 
FROM `bigquery-public-data.covid19_public_forecasts.county_14d` 
WHERE prediction_date >= "2021-09-01" 
"""


In [17]:
df = gbq.read_gbq(query+" limit 10")


Downloading: 100%|███████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 32.94rows/s]


In [12]:
data_path='../data/df_s.parquet'
df.to_parquet(data_path)

In [30]:
schema=pyarrow.parquet.read_schema(data_path)
print(schema)

county_name: string
county_fips_code: string
state_name: string
-- schema metadata --
pandas: '{"index_columns": [{"kind": "range", "name": null, "start": 0, "' + 633


In [23]:
bucket_name='beam-learn'
project_id='XXX'
region='us-central1'

In [26]:

beam_options=PipelineOptions(project=project_id, 
                             runner='DataflowRunner',
                             # runner='DirectRunner',
                             temp_location=f'gs://{bucket_name}/temp',
                             job_name = 'save-data-to-parquet',
                             region=region,
                             
                            )

file_path_prefix=f'gs://{bucket_name}/data/beam-output/small-parquet-pc/bq'
# file_path_prefix='../outputs/bqdata'

with beam.Pipeline(options=beam_options) as p:
    test_write = (
        p
        | 'Read BigQuery' >>  beam.io.ReadFromBigQuery( #beam.io.Read(beam.io.BigQuerySource(
            query=query+' limit 1000',
            use_standard_sql=True)
        #| beam.combiners.ToList()
        # Each row is a dictionary where the keys are the BigQuery columns
        | 'Save as parquet to GCS' >> beam.io.parquetio.WriteToParquet(file_path_prefix=file_path_prefix, 
                                           schema=pyarrow.schema(schema), 
                                           record_batch_size=1000,
                                           file_name_suffix='.parquet', num_shards=10))
        #| beam.combiners.ToList()
        #| beam.Map(print))


# Beam transformations

## Aggregations
### Group by and count
Doing the same group by 'ocean_proximity' and count operation using beam:

In [227]:
with beam.Pipeline() as p:
    near_ocean_count=(
        p 
        | beam.io.ReadFromText('../data/housing_short.csv', skip_header_lines=1)
        | beam.Map(lambda x: x.split(',')) # every step is a pCollection
        | beam.Map(lambda x: (x[9], 1))
        | beam.CombinePerKey(sum)
        | beam.io.WriteToText('../outputs/ocean_proximity_agg_count.txt')
    )


In [228]:
!cat '../outputs/ocean_proximity_agg_count.txt-00000-of-00001'


('<1H OCEAN', 51)
('NEAR OCEAN', 14)
('INLAND', 45)
('NEAR BAY', 9)
('ISLAND', 1)


### Filter and then count
Count when 'ocean_proximity' value is 'NEAR OCEAN'

In [229]:
# One can set runner='DirectRunner' or 'DataflowRunner'
with beam.Pipeline() as p:
    near_ocean_count=(
        p 
        | beam.io.ReadFromText('../data/housing_short.csv', skip_header_lines=1, min_bundle_size=1024)
        | beam.Map(lambda x: x.split(','))
        | beam.Filter(lambda x: x[9]=='NEAR OCEAN') # add filter here
        | beam.Map(lambda x: (x[9], 1))
        | beam.CombinePerKey(sum)
        | beam.io.WriteToText('../outputs/near_ocean_count.txt')
    )


In [230]:
# check newly generated output file
!cat '../outputs/near_ocean_count.txt-00000-of-00001'


('NEAR OCEAN', 14)


### Word count then sort

In [54]:
input_text = """
Peter Piper picked a peck of pickled peppers,
A peck of pickled peppers Peter Piper picked;
If Peter Piper picked a peck of pickled peppers,
Where’s the peck of pickled peppers Peter Piper picked?
"""
with open('../data/peter_piper.txt', 'w') as f:
    f.write(input_text)

In [55]:
cat ../data/peter_piper.txt


Peter Piper picked a peck of pickled peppers,
A peck of pickled peppers Peter Piper picked;
If Peter Piper picked a peck of pickled peppers,
Where’s the peck of pickled peppers Peter Piper picked?


In [119]:
def sort_by_key(record):
    record.sort(key=lambda x: x[0], reverse=False)
    return record

with beam.Pipeline() as p:
    word_count=(
        p 
        | 'Read input' >> beam.io.ReadFromText('../data/peter_piper.txt')
        | 'Split by space' >> beam.FlatMap(lambda x: x.split())
        | 'Strip symbols' >> beam.Map(lambda x: x.strip(',;?').lower())
        # | beam.Filter(lambda x: x=='Peter') # add filter here
        | beam.Map(lambda x: (x, 1))
        | beam.CombinePerKey(sum)
        | beam.combiners.ToList()
        | beam.Map(sort_by_key) # This works after flattening 
                                # by changing to list, beam.combiners.ToList():
                                # [[ele],[ele],[ele]] => [ele, ele, ele] 
                                # beam.Flatten() didn't work  
        | beam.io.WriteToText('../outputs/peter_count.txt')
    )


In [121]:
cat '../outputs/peter_count.txt-00000-of-00001'


[('a', 3), ('if', 1), ('of', 4), ('peck', 4), ('peppers', 4), ('peter', 4), ('picked', 4), ('pickled', 4), ('piper', 4), ('the', 1), ('where’s', 1)]


## Join: CoGroupByKey


In [79]:
p = beam.Pipeline()
movie_name = [
    (1, 'SpiderMan'),
    (2, 'Avenger'),
    (3, 'Titanic'),
    (4, 'Green Miles'),
]

movie_rating = [
    (1, 3.5),
    (2, 4.5),
    (3, 3.5),
    (4, 4.5),
]

name = p | 'Create Name Pcollection' >> beam.Create(movie_name)
rating = p | 'Create Rating Pcollection' >> beam.Create(movie_rating)

join_result = ({name, rating} | beam.CoGroupByKey()) | beam.Map(print)
p.run()

(1, (['SpiderMan'], [3.5]))
(2, (['Avenger'], [4.5]))
(3, (['Titanic'], [3.5]))
(4, (['Green Miles'], [4.5]))


## Partition
Split one Pcollection into many Pcollections

In [87]:
p = beam.Pipeline()
number = {1,2,3,4,5,6,7,8,9,10}

def partition_fn(element, num_partition):
    return element%num_partition

number_pc = p | beam.Create(number) | beam.Partition(partition_fn, 4)

number_pc[2] | 'Print the first Pcollection' >> beam.Map(print)

p.run()

2
6
10


## Flatten

Take many Pcollections and make them one Pcollection.

Note: The output has randomized orders of the input Pcollections. 

In [98]:
p = beam.Pipeline()

even={2,4,6,8}
odd = {1,3,5,7}
name = ('john', 'jim', 'mary')

even_pc = p | 'Create Pcollection from even numbers' >> beam.Create(even)
odd_pc = p | 'Create Pcollection from odd numbers' >> beam.Create(odd)
name_pc = p | 'Create Pcollection from names' >> beam.Create(name)

result = (even_pc, odd_pc, name_pc) | 'Flatten' >> beam.Flatten() | beam.Map(print)

p.run()


john
jim
mary
8
2
4
6
1
3
5
7


## Composite transformation

Combine multiple beam transformations in a single custom PTransform class. 
This is especially useful to avoid repeating codes.

For the previous example in 3.1.1:


In [231]:
# Old code 
with beam.Pipeline() as p:
    near_ocean_count=(
        p 
        | beam.io.ReadFromText('../data/housing_short.csv', skip_header_lines=1)
        | beam.Map(lambda x: x.split(',')) # output of every step is a pCollection
        | beam.Map(lambda x: (x[9], 1))
        | beam.CombinePerKey(sum)
        | beam.Map(print)
    )

('<1H OCEAN', 51)
('NEAR OCEAN', 14)
('INLAND', 45)
('NEAR BAY', 9)
('ISLAND', 1)


In [234]:
# New code packing several PTransforms inside of a class

class CombinedTransformation(beam.PTransform):
    # overwrite the expand() method
    def expand(self, input_pcollection):
        a = (
            input_pcollection
                                | beam.Map(lambda x: x.split(',')) # every step is a pCollection
                                | beam.Map(lambda x: (x[9], 1))
                                | beam.CombinePerKey(sum) 
        )
        return a
    

with beam.Pipeline() as p:
    near_ocean_count=(
        p 
        | beam.io.ReadFromText('../data/housing_short.csv', skip_header_lines=1)
        | "Combined PTransform" >> CombinedTransformation()
        | beam.Map(print)
    )

('<1H OCEAN', 51)
('NEAR OCEAN', 14)
('INLAND', 45)
('NEAR BAY', 9)
('ISLAND', 1)


# ParDo transformations
Transformations such as Map and FlatMap are "ParDo"s. 

Write custom ParDo by `beam.ParDo( custome_dofn_class, arg=arg)`, obviously you need to write your own `DoFn` class `custome_dofn_class`.

To write custome `DoFn`, overwrite the `process` function.

## Pardo for Map: Housing data

In [239]:
# Pardo for Map
# FlatMap: DoFn returns list

class SplitRow(beam.DoFn):
    def process(self, element):
        return [element.split(',')]

class FilterItems(beam.DoFn):
    def process(self, element):
        if element[9]=='NEAR OCEAN' and float(element[8])<100000:
            return [element]

        
with beam.Pipeline() as p:
    near_ocean_count=(
        p 
        | "Read File" >> beam.io.ReadFromText('../data/housing_short.csv', skip_header_lines=1)
        | "Split" >> beam.ParDo(SplitRow())
        | "Filter Customer" >> beam.ParDo(FilterItems())
        | "Print" >> beam.Map(print)
    )
    


['-124.06', '40.86', '34.0', '4183.0', '', '1891.0', '669.0', '3.2216', '98100.0', 'NEAR OCEAN']


## ParDo for FlatMap: Wordcount revisit

In [172]:
# Pardo for FlatMap
# FlatMap: DoFn returns element
# Use generator to unflatten

class SplitRow(beam.DoFn):
    def process(self, element):
        return element.split()
    
class StripLower(beam.DoFn):
    def process(self, element):
        return [element.strip(',;?').lower()]
    
def find_word(element):
    words = ['peter', 'piper', 'picked']
    if element in words:
        return True

def sort_by_key(record):
    """make into generator
    """
    record.sort(key=lambda x: x[0], reverse=False)
    for x in record:
        yield x
        
class SumValuesByKey(beam.DoFn):
    def process(self, element):
        (key, value) = element
        return [(key, sum(value))]

with beam.Pipeline() as p:
    word_count=(
        p 
        | "Read file" >> beam.io.ReadFromText('../data/peter_piper.txt')
        | "Split row - flatmap" >> beam.ParDo(SplitRow()) # FlatMap
        | "Strip and lower words - map" >> beam.ParDo(StripLower()) # Map
        | "Find word" >> beam.Filter(find_word) # Map        
        | "Attach 1 count" >> beam.Map(lambda x: (x, 1))
        | "Make list of ones per word" >> beam.GroupByKey() 
                            # step 1 of old beam.CombinePerKey(sum)
                            # outputs:
                                    # ('peter', [1,1,1,1])
                                    # ('picked', [1,1,1,1])
                                    # ('piper', [1,1,1,1])
        | "Sum the ones per key" >> beam.ParDo(SumValuesByKey()) 
                            # step 2 of old beam.CombinePerKey(sum)
                            # outputs:
                                    # ('peter', 4)
                                    # ('picked', 4)
                                    # ('piper', 4)        
        | "Turn into list" >> beam.combiners.ToList()
        | "Sort by key" >> beam.FlatMap(sort_by_key) # Use Generator to "unflatten" the list. 
                                    # This solves the problem in previous section 4.1.3
        | "Print" >> beam.Map(print)
    )



('peter', 4)
('picked', 4)
('piper', 4)


# Side inputs and side outputs in ParDo
## Side inputs in Pardo
To use side inputs in ParDo, modify the corresponding DoFn to read side inputs. 

Example using housing data to attach a labe by `ocean_proximity`.

In [241]:
# side_input is labels
# alternative side_input can be a list of items ids that should be excluded etc.
# Or overwrite part of the items etc.

labels=[
('NEAR BAY', 'premium'),
('<1H OCEAN', 'good'),
('INLAND', 'common'),
('NEAR OCEAN', 'premium'),
('ISLAND', 'super premium')]


class SplitRow(beam.DoFn):
    def process(self, element, side_input):
        rows = element.split(',') 
        return [rows + [dict( tuple(labels))[rows[9]] ]] 


class FilterItems(beam.DoFn):
    def process(self, element):
        if element[9]=='NEAR OCEAN' and float(element[8])<100000:
            return [element]


        
with beam.Pipeline() as p:
    near_ocean_count=(
        p 
        | "Read File" >> beam.io.ReadFromText('../data/housing_short.csv', skip_header_lines=1)
        | "Split" >> beam.ParDo(SplitRow(), side_input=labels)
        | "Filter Customer" >> beam.ParDo(FilterItems())
        | "Print" >> beam.Map(print)
    )
    

['-124.06', '40.86', '34.0', '4183.0', '', '1891.0', '669.0', '3.2216', '98100.0', 'NEAR OCEAN', 'premium']


## Side outputs in ParDo

In [218]:

funny_words = ['peter', 'piper', 'picked']


class ProcessWords(beam.DoFn):
    def process(self, element, side_input):
        if element[0] in side_input:
            yield element
        else:
            yield beam.pvalue.TaggedOutput('boring', element)

with beam.Pipeline() as p:
    word_count=(
        p 
        | "Read file" >> beam.io.ReadFromText('../data/peter_piper.txt')
        | "Split row - flatmap" >> beam.FlatMap(lambda x: x.split()) # FlatMap
        | "Strip and lower words - map" >> beam.Map(lambda x: x.strip(',;?').lower()) # Map
        | "Attach 1 count" >> beam.Map(lambda x: (x, 1))
        | "Count frequency per word" >> beam.CombinePerKey(sum) 
        | "Section word count" >> beam.ParDo(ProcessWords(), side_input=funny_words).with_outputs('boring', main='funny')
    ) 

    # funny tagged
    word_count.funny | 'Write funny wordcounts' >> beam.io.WriteToText('../outputs/funny_wordcount.txt')
    # boring tagged
    word_count.boring | 'Write boring wordcounts' >> beam.io.WriteToText('../outputs/boring_wordcount.txt')

    


In [219]:
cat '../outputs/funny_wordcount.txt-00000-of-00001'

('peter', 4)
('piper', 4)
('picked', 4)


In [220]:
cat '../outputs/boring_wordcount.txt-00000-of-00001'

('a', 3)
('peck', 4)
('of', 4)
('pickled', 4)
('peppers', 4)
('if', 1)
('where’s', 1)
('the', 1)
